In [16]:
from Bio import SeqIO, AlignIO, Phylo
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from collections import Counter
import os
import matplotlib.pyplot as plt

def process_gene(gene_name, input_fasta):
    # Criar diretório de saída
    output_dir = "Resultados da Analise da Arvore Filogenetica"
    os.makedirs(output_dir, exist_ok=True)

    # Leitura do arquivo fasta
    with open(input_fasta) as handle:
        records = list(SeqIO.parse(handle, format='fasta'))

    # Garantir que temos exatamente 7 sequências
    if len(records) < 7:
        raise ValueError(f"O arquivo FASTA deve conter pelo menos 7 sequências. Encontradas: {len(records)}")

    # Extrair as primeiras 7 sequências e criar SeqRecords com IDs personalizados
    seq_records = [SeqRecord(record.seq, id=f'{gene_name}_{i+1}') 
                   for i, record in enumerate(records[:7])]

    # Criar o alinhamento
    alignment = MultipleSeqAlignment(seq_records)

    # Verificar IDs duplicados no alinhamento múltiplo
    id_counts = Counter(record.id for record in alignment)
    duplicates = [id_ for id_, count in id_counts.items() if count > 1]
    if duplicates:
        print(f"IDs duplicados detectados: {duplicates}")
        raise ValueError("Por favor, corrija os IDs duplicados antes de continuar.")

    # Salvar o alinhamento
    alignment_file = os.path.join(output_dir, f'{gene_name}_alignment.fasta')
    AlignIO.write(alignment, alignment_file, 'fasta')

    # Calcular matriz de distância
    calculator = DistanceCalculator('blosum62')
    dm = calculator.get_distance(alignment)
    
    # Imprimir a matriz de distância
    print(f"Matriz de Distância:\n{dm}")

    # Construir árvores
    constructor = DistanceTreeConstructor()
    upgma_tree = constructor.upgma(dm)
    nj_tree = constructor.nj(dm)

    # Remover rótulos 'Inner' dos clados e destacar clados com o gênero "Homo"
    for tree in [upgma_tree, nj_tree]:
        for clade in tree.find_clades():
            if 'Inner' in (clade.name or ''):
                clade.name = ""
            if 'Homo' in (clade.name or ''):
                clade.color = 'orange'
                clade.name = '*** ' + clade.name + ' ***'  # Marca o clado

    # Salvar árvores
    Phylo.write(upgma_tree, os.path.join(output_dir, f'{gene_name}_upgma_tree.nwk'), 'newick')
    Phylo.write(nj_tree, os.path.join(output_dir, f'{gene_name}_nj_tree.nwk'), 'newick')

    # Processar e desenhar árvores
    for tree, tree_type in [(upgma_tree, 'UPGMA'), (nj_tree, 'NJ')]:
        plt.figure(figsize=(10, 8))
        Phylo.draw(tree, do_show=False)
        plt.title(f'{gene_name} - {tree_type} Tree')
        tree_img = os.path.join(output_dir, f'{gene_name}_{tree_type}_tree.png')
        plt.savefig(tree_img)
        plt.close()

    print(f"Processamento concluído para o gene {gene_name}. Resultados salvos em '{output_dir}'.")


#### **1: Gene ptsP**

In [17]:
gene_name = "ptsp"
input_fasta = "genes/ptsP_blast.fasta"

process_gene(gene_name, input_fasta)

Matriz de Distância:
ptsp_1  0.000000
ptsp_2  0.001080    0.000000
ptsp_3  0.001080    0.002159    0.000000
ptsp_4  0.001439    0.002519    0.002519    0.000000
ptsp_5  0.002157    0.003236    0.003236    0.003596    0.000000
ptsp_6  0.001439    0.000360    0.002519    0.002879    0.003596    0.000000
ptsp_7  0.001799    0.002879    0.002879    0.003239    0.003955    0.003239    0.000000
    ptsp_1  ptsp_2  ptsp_3  ptsp_4  ptsp_5  ptsp_6  ptsp_7
Processamento concluído para o gene ptsp. Resultados salvos em 'Resultados da Analise da Arvore Filogenetica'.


<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

#### **2: Gene ButyrylCoA**

In [18]:
gene_name = "butyrylCoA"
input_fasta = "genes/butyrylCoA_blast.fasta"

process_gene(gene_name, input_fasta)

Matriz de Distância:
butyrylCoA_1    0.000000
butyrylCoA_2    0.002124    0.000000
butyrylCoA_3    0.002123    0.004246    0.000000
butyrylCoA_4    0.000000    0.002124    0.002123    0.000000
butyrylCoA_5    0.003398    0.001700    0.005520    0.003398    0.000000
butyrylCoA_6    0.003398    0.001276    0.005520    0.003398    0.002975    0.000000
butyrylCoA_7    0.003398    0.001276    0.005520    0.003398    0.002975    0.002551    0.000000
    butyrylCoA_1    butyrylCoA_2    butyrylCoA_3    butyrylCoA_4    butyrylCoA_5    butyrylCoA_6    butyrylCoA_7
Processamento concluído para o gene butyrylCoA. Resultados salvos em 'Resultados da Analise da Arvore Filogenetica'.


<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>